In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [22]:
titanic = sns.load_dataset('titanic')

In [23]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [24]:
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [25]:
titanic.drop(['age','deck'], axis=1, inplace=True)

In [26]:
titanic.isnull().sum()

survived       0
pclass         0
sex            0
sibsp          0
parch          0
fare           0
embarked       2
class          0
who            0
adult_male     0
embark_town    2
alive          0
alone          0
dtype: int64

In [27]:
titanic.shape

(891, 13)

In [28]:
titanic = titanic.dropna()

In [29]:
titanic.isnull().sum()

survived       0
pclass         0
sex            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [30]:
data = titanic[['pclass','sex','sibsp','parch','embarked','who','alone']].copy()

In [31]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,male,1,0,S,man,False
1,1,female,1,0,C,woman,False
2,3,female,0,0,S,woman,True
3,1,female,1,0,S,woman,False
4,3,male,0,0,S,man,True


In [32]:
sex = {'male':0, 'female':1}
data['sex'] = data['sex'].map(sex)
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,S,man,False
1,1,1,1,0,C,woman,False
2,3,1,0,0,S,woman,True
3,1,1,1,0,S,woman,False
4,3,0,0,0,S,man,True


In [33]:
data['embarked'].value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [34]:
embarked = {'S':0,'C':1,'Q':2}
data['embarked'] = data['embarked'].map(embarked)
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,0,man,False
1,1,1,1,0,1,woman,False
2,3,1,0,0,0,woman,True
3,1,1,1,0,0,woman,False
4,3,0,0,0,0,man,True


In [35]:
data['who'].value_counts()

man      537
woman    269
child     83
Name: who, dtype: int64

In [36]:
who = {'man':0,'woman':1,'child':2}
data['who'] = data['who'].map(who)
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,0,0,False
1,1,1,1,0,1,1,False
2,3,1,0,0,0,1,True
3,1,1,1,0,0,1,False
4,3,0,0,0,0,0,True


In [37]:
data['alone'].value_counts()

True     535
False    354
Name: alone, dtype: int64

In [38]:
alone = {False:0,True:1}
data['alone'] = data['alone'].map(alone)
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,0,0,0
1,1,1,1,0,1,1,0
2,3,1,0,0,0,1,1
3,1,1,1,0,0,1,0
4,3,0,0,0,0,0,1


In [40]:
X = data.copy()
y = titanic['survived']
X.shape, y.shape

((889, 7), (889,))

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [42]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((711, 7), (178, 7), (711,), (178,))

### Estimation of coefficients of Linear Regression

In [61]:
sel = SelectFromModel(LinearRegression())

In [62]:
sel.fit(X_train, y_train)

SelectFromModel(estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                           n_jobs=None, normalize=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [63]:
sel.get_support()

array([ True,  True, False, False, False,  True, False])

In [64]:
sel.estimator_.coef_

array([-0.1465448 ,  0.32116345, -0.08154868, -0.05642944,  0.06482235,
        0.24199831, -0.09299509])

In [65]:
mean = np.mean(np.abs(sel.estimator_.coef_))

In [66]:
mean

0.14364315926685428

In [67]:
np.abs(sel.estimator_.coef_)

array([0.1465448 , 0.32116345, 0.08154868, 0.05642944, 0.06482235,
       0.24199831, 0.09299509])

In [68]:
features = X_train.columns[sel.get_support()]
features

Index(['pclass', 'sex', 'who'], dtype='object')

In [53]:
X_train_reg = sel.transform(X_train)
X_test_reg=  sel.transform(X_test)

In [55]:
X_train_reg.shape, X_test_reg.shape

((711, 3), (178, 3))

In [58]:
def random_forest(X_train, X_test, y_train, y_test):
    model = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('Accuracy : ', accuracy_score(y_test, y_pred))

In [59]:
%%time
random_forest(X_train_reg, X_test_reg, y_train, y_test)

Accuracy :  0.7415730337078652
Wall time: 259 ms


In [60]:
%%time
random_forest(X_train, X_test, y_train, y_test)

Accuracy :  0.7359550561797753
Wall time: 260 ms


### Logistic Regression Coefficient with L1 Regularization

In [76]:
sel = SelectFromModel(LogisticRegression(penalty='l1', C = 0.05, 
                                         solver = 'liblinear'))
sel.fit(X_train, y_train)
sel.get_support()

array([ True,  True,  True, False,  True,  True, False])

In [77]:
sel.estimator_.coef_

array([[-0.63178281,  1.11135677, -0.20115027,  0.        ,  0.07345507,
         1.01851956,  0.        ]])

In [78]:
X_train_l1 = sel.transform(X_train)
X_test_l1 = sel.transform(X_test)

In [79]:
%%time
random_forest(X_train_l1, X_test_l1, y_train, y_test)

Accuracy :  0.7415730337078652
Wall time: 259 ms


### L2 Regularization